In [4]:
import cv2
import numpy as np

def manual_canny(image, low_threshold, high_threshold):
    """
    Manual implementation of the Canny Edge Detection algorithm.
    
    :param image: Input grayscale image.
    :param low_threshold: Lower bound for hysteresis thresholding.
    :param high_threshold: Upper bound for hysteresis thresholding.
    :return: Edge-detected image.
    """
    # Step 1: Gaussian Blurring
    blurred = cv2.GaussianBlur(image, (5, 5), 1.4)

    # Step 2: Compute Gradients (Sobel Filters)
    grad_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    gradient_direction = np.arctan2(grad_y, grad_x) * 180 / np.pi
    gradient_direction = (gradient_direction + 180) % 180  # Normalize to [0, 180)

    # Step 3: Non-Maximum Suppression
    suppressed = non_max_suppression(gradient_magnitude, gradient_direction)

    # Step 4: Double Thresholding
    strong_edges, weak_edges = double_threshold(suppressed, low_threshold, high_threshold)

    # Step 5: Edge Tracking by Hysteresis
    edges = edge_tracking_by_hysteresis(strong_edges, weak_edges)

    return edges

def non_max_suppression(gradient_magnitude, gradient_direction):
    """
    Perform Non-Maximum Suppression to thin edges.
    """
    rows, cols = gradient_magnitude.shape
    suppressed = np.zeros((rows, cols), dtype=np.float32)
    angle = gradient_direction

    for i in range(1, rows-1):
        for j in range(1, cols-1):
            try:
                # Quantize direction
                q = 255
                r = 255
                if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                    q = gradient_magnitude[i, j+1]
                    r = gradient_magnitude[i, j-1]
                elif 22.5 <= angle[i, j] < 67.5:
                    q = gradient_magnitude[i+1, j-1]
                    r = gradient_magnitude[i-1, j+1]
                elif 67.5 <= angle[i, j] < 112.5:
                    q = gradient_magnitude[i+1, j]
                    r = gradient_magnitude[i-1, j]
                elif 112.5 <= angle[i, j] < 157.5:
                    q = gradient_magnitude[i-1, j-1]
                    r = gradient_magnitude[i+1, j+1]

                # Suppress non-maximum pixels
                if gradient_magnitude[i, j] >= q and gradient_magnitude[i, j] >= r:
                    suppressed[i, j] = gradient_magnitude[i, j]
                else:
                    suppressed[i, j] = 0
            except IndexError as e:
                pass

    return suppressed

def double_threshold(image, low_threshold, high_threshold):
    """
    Apply double thresholding to identify strong and weak edges.
    """
    strong_edges = (image >= high_threshold).astype(np.uint8)
    weak_edges = ((image >= low_threshold) & (image < high_threshold)).astype(np.uint8)
    return strong_edges, weak_edges

def edge_tracking_by_hysteresis(strong_edges, weak_edges):
    """
    Perform edge tracking by hysteresis to finalize edges.
    """
    rows, cols = strong_edges.shape
    edges = np.copy(strong_edges)

    for i in range(1, rows-1):
        for j in range(1, cols-1):
            if weak_edges[i, j] == 1:
                # If a weak edge is connected to a strong edge, mark it as an edge
                if np.any(strong_edges[i-1:i+2, j-1:j+2]):
                    edges[i, j] = 1

    return (edges * 255).astype(np.uint8)

# Main Code
image_path = "/home/student/Pictures/Screenshots/Screenshot from 2024-12-03 12-23-34.png"  # Replace with your image path
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

if image is None:
    print("Error: Unable to load image. Please check the path.")
else:
    edges = manual_canny(image, low_threshold=50, high_threshold=150)

    # Display the results
    cv2.imshow("Original Image", image)
    cv2.imshow("Edge-Detected Image (Manual Canny)", edges)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
